In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np 
import pandas as pd
import os
import scipy.stats as ss
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import f_oneway
import pandas as pd
import datetime
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Let's start by loading the datasets

In [4]:
# Merge the train and store dataframes
train_df = pd.merge(train_df, store_df, on='Store', how='left')

# Merge the test and store dataframes
test_df = pd.merge(test_df, store_df, on='Store', how='left')

# Display the first few rows of each merged dataframe
train_df.head(), test_df.head()

past_sales = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/rossmann.csv')
store = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/store.csv')

# 데이터 제거
store = store.drop(['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1)

# 데이터 병합
data = pd.merge(left = past_sales,right = store, on = 'Store', how = 'right')
data

C:\Users\rlaeo\AppData\Local\Temp\ipykernel_8584\3219985095.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  past_sales = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/rossmann.csv')


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,0
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,0
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,0
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,1
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,1
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,1
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,1


In [19]:
a = data.groupby('StoreType').size()
a

StoreType
a    551627
b     15830
c    136840
d    312912
dtype: int64